# Reviewing and Preparing the Data
- Proceedeeding column by column, review data and take necessary stepts to parse/clean/standardize values

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import processed dataset
auctions = pd.read_csv(
    'data/auction_data_processed.csv', 
    usecols=[0, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14]
)

auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in
0,Pablo PICASSO,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN
1,Pablo PICASSO,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN
2,Pablo PICASSO,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN
3,Pablo PICASSO,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN
4,Pablo PICASSO,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN


## 1. `artist_name`
- Convert to title case
### 1.1 Convert to title case

In [3]:
auctions['artist_name'] = auctions['artist_name'].str.title()
auctions['artist_name'].sample(10)

27476             Zhou Chunya
15555           Lucio Fontana
1813     Jean-Michel Basquiat
28813          Rudolf Stingel
29681            Damien Hirst
23745        Christopher Wool
45685          Georges Braque
35311          Bernard Buffet
38700          Zhang Xiaogang
28933          Rudolf Stingel
Name: artist_name, dtype: object

## 2. Merge with MoMA's `artist` Dataset for Additional Features

- Load MoMA `Artists` dataset, specifying that 0 be counted as NaN since I know that `EndDate` (i.e., the year of the artist's death) is sometimes set to 0 when not relevant (artist still living) or not known
- Standardize `auctions['artist_name']` and `Artists['DisplayName']` using `.str.lower()`
- Merge with `auctions` and `Artists` on lowercase artist name

In [4]:
# Load dataset
moma_artists = pd.read_csv('../../data-exploration/data/moma/Artists.csv', usecols=[1, 3, 4, 5, 6], na_values=[0])
moma_artists.head()

,DisplayName,Nationality,Gender,BeginDate,EndDate
0,Robert Arneson,American,Male,1930.0,1992.0
1,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,Bill Arnold,American,Male,1941.0,NaN
3,Charles Arnoldi,American,Male,1946.0,NaN
4,Per Arnoldi,Danish,Male,1941.0,NaN


In [5]:
# Standardize artist names from both datasets

from unidecode import unidecode

# Ensure that strings are stripped of accents to ensure better matching
def standardize_name(name):
    return unidecode(name.lower())

moma_artists['name'] = moma_artists['DisplayName'].apply(standardize_name)
auctions['name'] = auctions['artist_name'].apply(standardize_name)

In [6]:
# Merge
auctions = pd.merge(
    left=auctions, 
    right=moma_artists,
    how='left',
    left_on='name',
    right_on='name'
)
auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,name,DisplayName,Nationality,Gender,BeginDate,EndDate
0,Pablo Picasso,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
1,Pablo Picasso,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
2,Pablo Picasso,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
3,Pablo Picasso,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0
4,Pablo Picasso,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN,pablo picasso,Pablo Picasso,Spanish,Male,1881.0,1973.0


## 3. `StartDate` and `EndDate`

### 3.1 Confirm Dtypes

In [7]:
auctions['BeginDate'].dtype

dtype('float64')

In [8]:
auctions['EndDate'].dtype

dtype('float64')

### 3.2 Look for missings
It's expected to be missing values for `EndDate` if the artist is still alive, but if we're missing values for `StartDate` there's likely a problem. For each artist missing `StartDate` information, I'll need to manually add these.

In [9]:
auctions[auctions['BeginDate'].isna()]['artist_name'].unique()

array(['Qi Baishi', 'Fu Baoshi', 'Wu Guanzhong', 'Sanyu', 'Banksy',
       'Wassily Kandinsky', 'Ed Ruscha', 'Chu Teh-Chun', 'Pan Tianshou',
       'Francois-Xavier Lalanne', 'Kaws', 'Lin Fengmian', 'Edgar Degas',
       'Pu Ru', 'Zhou Chunya', 'Zeng Fanzhi', 'Liu Ye',
       'Tsuguharu Foujita', 'Norman Rockwell', 'Zhang Xiaogang',
       'Dong Qichang', 'Wu Hufan', 'Tamara De Lempicka', 'Wen Zhengming',
       'Kenny Scharf'], dtype=object)

In [10]:
# Specify missing data as dictionary
artist_dates = {
    'Qi Baishi': {'BeginDate':1864, 'EndDate':1957, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Fu Baoshi': {'BeginDate':1904, 'EndDate':1965, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Wu Guanzhong': {'BeginDate':1919, 'EndDate':2010, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Sanyu':{'BeginDate':1901, 'EndDate':1966, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Banksy':{'BeginDate':1974, 'Nationality':'English', 'Nationality':'English', 'Gender':'Male'},
    'Wassily Kandinsky':{'BeginDate':1866, 'EndDate':1944, 'Nationality':'Russian', 'Gender':'Male'}, 
    'Ed Ruscha':{'BeginDate':1937, 'Nationality':'American', 'Gender':'Male'}, 
    'Chu Teh-Chun':{'BeginDate':1920, 'EndDate':2014, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Pan Tianshou':{'BeginDate':1897, 'EndDate':1971, 'Nationality':'Chinese', 'Gender':'Male'},
    'Francois-Xavier Lalanne':{'BeginDate':1927, 'EndDate':2008, 'Nationality':'French', 'Gender':'Male'}, 
    'Kaws':{'BeginDate':1974, 'Nationality':'American', 'Gender':'Male'}, 
    'Lin Fengmian':{'BeginDate':1900, 'EndDate':1991, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Edgar Degas':{'BeginDate':1834, 'EndDate':1917, 'Nationality':'French', 'Gender':'Male'},
    'Pu Ru':{'BeginDate':1896, 'EndDate':1963, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Zhou Chunya':{'BeginDate':1955, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Zeng Fanzhi':{'BeginDate':1964, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Liu Ye':{'BeginDate':1964, 'Nationality':'Chinese', 'Gender':'Male'},
    'Tsuguharu Foujita':{'BeginDate':1886, 'EndDate':1968, 'Nationality':'Japanese', 'Gender':'Male'}, 
    'Norman Rockwell':{'BeginDate':1894, 'EndDate':1978, 'Nationality':'American', 'Gender':'Male'}, 
    'Zhang Xiaogang':{'BeginDate':1958, 'Nationality':'Chinese', 'Gender':'Male'},
    'Dong Qichang':{'BeginDate':1555, 'EndDate':1636, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Wu Hufan':{'BeginDate':1894, 'EndDate':1968, 'Nationality':'Chinese', 'Gender':'Male'}, 
    'Tamara De Lempicka':{'BeginDate':1898, 'EndDate':1980, 'Nationality':'Polish', 'Gender':'Female'}, 
    'Wen Zhengming':{'BeginDate':1470, 'EndDate':1559, 'Nationality':'Chinese', 'Gender':'Male'},
    'Kenny Scharf':{'BeginDate':1958, 'Nationality':'American', 'Gender':'Male'}
}

# Add missing data to DataFrame
for name, values in artist_dates.items():
    current_artist = (auctions['artist_name'] == name)

    auctions.loc[current_artist, 'BeginDate'] = values.get('BeginDate', '')
    auctions.loc[current_artist, 'EndDate'] = values.get('EndDate', np.nan)
    auctions.loc[current_artist, 'Nationality'] = values.get('Nationality', '')
    auctions.loc[current_artist, 'Gender'] = values.get('Gender', '')

Last I want to rename these columns.

In [11]:
auctions.rename(columns={'BeginDate':'birth_year', 'EndDate':'death_year'}, inplace=True)

## 4. Drop Redundant Column Post-Merge

In [12]:
auctions.drop(columns=['DisplayName', 'name'], inplace=True)
auctions.head()

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year
0,Pablo Picasso,Fillette au bateau (Maya),", 1938",oil on canvas,73.3 x 60 cm,"Mar 1, 2023",Sotheby's• London,Modern & Contemporary Evening Auction,Lot6002,NaN,NaN,NaN,Spanish,Male,1881.0,1973.0
1,Pablo Picasso,Homme assis,", 1969",oil on panel laid down on cradled panel,28.7 x 56.6 cm,"Nov 30, 2022",Christie's,Live Auction 19901 20th/21st Century Art Eveni...,Lot35,"HK\$10,650,000• US\$1,363,489","HK\$8,200,000–HK\$12,800,000(est)",NaN,Spanish,Male,1881.0,1973.0
2,Pablo Picasso,Buffalo Bill,", 1911",oil and sand on canvas,33.3 x 46.3 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot12,"US\$12,412,500","US\$10,000,000–US\$15,000,000(est)",NaN,Spanish,Male,1881.0,1973.0
3,Pablo Picasso,Homme à la moustache,", 1970",oil and oil stick on panel,65.1 x 129.4 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot30,"US\$4,620,000","US\$4,000,000–US\$6,000,000(est)",NaN,Spanish,Male,1881.0,1973.0
4,Pablo Picasso,Le peintre et son modèle,", 1964",oil and ripolin on canvas,195.0 x 130.0 cm,"Nov 17, 2022",Christie's,Live Auction 20988 20th Century Evening Sale,Lot36,"US\$10,351,500","US\$8,000,000–US\$12,000,000(est)",NaN,Spanish,Male,1881.0,1973.0


## 5. Parse `auction_date` as DateTime object

Jumping out of order for a moment since the auction date may help eliminate problematic artwork dates (i.e., an artwork completed after it was auctioned, etc.)

In [13]:
# Look for any issues
auctions['auction_date_parsed'] = pd.to_datetime(auctions['auction_date'])
auctions[['auction_date', 'auction_date_parsed']].sample(10)

,auction_date,auction_date_parsed
12224,"May 16, 2017",2017-05-16
33616,"Jul 10, 2020",2020-07-10
23318,"May 18, 2007",2007-05-18
38585,"May 5, 1993",1993-05-05
32104,"Jun 30, 1999",1999-06-30
35319,"Jun 19, 2007",2007-06-19
10972,"Nov 5, 2014",2014-11-05
26499,"Nov 2, 1993",1993-11-02
31559,"Jul 9, 2020",2020-07-09
28266,"Oct 4, 2019",2019-10-04


## 6. Parse `date`
Because of how I scraped this data, these values should all be prepended by `, `, so I'll start by stripping this.

### 6.1 Strip commas and spaces

In [14]:
auctions['date'] = auctions['date'].str.lstrip(', ')
auctions['date'].sample(10)

15356         1954
31736          NaN
39792         1900
52518         1966
44309         1988
19254    1841-1919
27222          NaN
48192         2000
32445         1982
14921         1981
Name: date, dtype: object

### 6.2 Examine Formats
Now let's get a sense for the various date formats and parse each one.

In [15]:
# Review `date` values that are not YYYY format
format_oneyear = (auctions['date'].str.match(r'^\d{4}$').fillna(False))

auctions.loc[~format_oneyear, 'date'].dropna().sample(10)

5752     1840-1926
26058    1830-1903
34940    1928-1999
12111    1887-1985
472      1881-1973
38451    1928-2011
2217     1928-1987
3615     1928-1987
4239     1928-1987
2270     1928-1987
Name: date, dtype: object

In [16]:
# Review `date` values that are additionally not YYYY-YYY (or similar) format
format_span = (auctions['date'].str.match(r'^\d{4} ?[-–/]? ?\d{4}$').fillna(False))

auctions.loc[~format_oneyear & ~format_span, 'date'].dropna().sample(10)

19128         circa 1900
25880         circa 1875
33406         1924, 1913
11614        circa 1950s
25896         circa 1886
18386    circa 1904-1906
11998         circa 1980
36641    circa 1967-1969
12078         circa 1966
11979    circa 1980-1982
Name: date, dtype: object

In [17]:
# Review `date` values that additionally are not prepended by 'circa'
format_circa = (auctions['date'].str.match(r'^circa ?\d{4} ?[-–/]? ?(\d{4})?$').fillna(False))

auctions.loc[~format_oneyear & ~format_span & ~format_circa, 'date'].dropna()

11600                     1940s
11614               circa 1950s
11621                     1940s
11623               1940s-1950s
14308    2005, 2005, 2005, 2005
33406                1924, 1913
33728               circa 1920s
46679                2002, 2009
50837      1936-1937, 1936-1937
Name: date, dtype: object

So it seems we're dealing with three main formats:
- A standard 4 digit year
- A spanned date consisting of two four-digit years separated by hyphens, slashes or spaces
- An approximate year or year span prepended by 'circa' and/or appended with 's'

So to parse these dates, I'm going to extract the first four-digit number as `start_date` and the last four-digit number as `end_date`, which seems like it should cover all the scenarios here. Then I'll make sure that there's no situation where `start_date > end_date`.

### 6.1 Parse

In [18]:
# Extract artwork start date
auctions['start_date'] = auctions['date'].str.extract(r'^\D*?(\d{4}).*?').astype('float')

# Extract artwork end date
auctions['end_date'] = auctions['date'].str.extract(r'^.*?(\d{4})\D*?$').astype('float')

cols = ['date', 'start_date', 'end_date']
auctions.loc[auctions['date'].notna(), cols].sample(10)

,date,start_date,end_date
17921,1940,1940.0,1940.0
43350,1966,1966.0,1966.0
45117,1996,1996.0,1996.0
50147,1983,1983.0,1983.0
29209,2008,2008.0,2008.0
1274,1984,1984.0,1984.0
34406,1928-1999,1928.0,1999.0
30614,1924,1924.0,1924.0
47607,2007,2007.0,2007.0
36666,1977,1977.0,1977.0


So far this looks great. I just want to check the edge cases above.

In [19]:
auctions.loc[~format_oneyear & ~format_span & ~format_circa, cols].dropna()

,date,start_date,end_date
11600,1940s,1940.0,1940.0
11614,circa 1950s,1950.0,1950.0
11621,1940s,1940.0,1940.0
11623,1940s-1950s,1940.0,1950.0
14308,"2005, 2005, 2005, 2005",2005.0,2005.0
33406,"1924, 1913",1924.0,1913.0
33728,circa 1920s,1920.0,1920.0
46679,"2002, 2009",2002.0,2009.0
50837,"1936-1937, 1936-1937",1936.0,1937.0


Looks good! But actually not quite right. The value `1940s` for instance should correspond not to a start and end date of 1940 but a span of 1940-1949. Let me specifically look for dates that end in 's' and see what we're dealing with here.

In [20]:
approx_formats = (auctions['date'].str.match(r'.*?s.*?').fillna(False))

auctions.loc[approx_formats, 'date']

11600          1940s
11614    circa 1950s
11621          1940s
11623    1940s-1950s
33728    circa 1920s
Name: date, dtype: object

There's only a few of these, so I'll just correct them by hand.

In [21]:
auctions.loc[11600, ['start_date', 'end_date']] = [1940.0, 1949.0]
auctions.loc[11614, ['start_date', 'end_date']] = [1950.0, 1959.0]
auctions.loc[11621, ['start_date', 'end_date']] = [1940.0, 1949.0]
auctions.loc[11623, ['start_date', 'end_date']] = [1940.0, 1959.0]
auctions.loc[33728, ['start_date', 'end_date']] = [1920.0, 1929.0]

auctions.loc[approx_formats, cols]

,date,start_date,end_date
11600,1940s,1940.0,1949.0
11614,circa 1950s,1950.0,1959.0
11621,1940s,1940.0,1949.0
11623,1940s-1950s,1940.0,1959.0
33728,circa 1920s,1920.0,1929.0


### 6.4 Look for weirdness: Situations where `start_date > end_date`

In [22]:
weird = (auctions['start_date'] > auctions['end_date'])

auctions[weird]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
33406,Ernst Ludwig Kirchner,Sertigweg im Sommer (recto); Fehmarnküste mit ...,"1924, 1913",oil on canvas,47 ¼ x 35 5/8 in. (120.2 x 90.5 cm.),"Jun 30, 2021",Christie's,20th/21st Century: London Evening Sale,Lot48,"£982,500• US\$1,358,630","£800,000–£1,200,000(est)",NaN,German,Male,1880.0,1938.0,2021-06-30,1924.0,1913.0
41636,Francis Picabia,Nature morte à la soupière,1979-1953,oil on canvas,73.3 x 92.4 cm,"May 9, 2013","Christie's • New York, Rockefeller Center",Impressionist & Modern Day Sale,Lot238,"US\$159,750","US\$80,000–US\$120,000(est)",NaN,French,Male,1879.0,1953.0,2013-05-09,1979.0,1953.0


In [23]:
# Correct weirdness
auctions.loc[33406, ['start_date', 'end_date']] = [1913.0, 1924.0]
auctions.loc[41636, ['start_date', 'end_date']] = [1908.0, 1908.0]

auctions.loc[weird, cols]

,date,start_date,end_date
33406,"1924, 1913",1913.0,1924.0
41636,1979-1953,1908.0,1908.0


### 6.5 Look for weirdness: Situations where `end_date > death_yer` or `start_date < birth_year`

In [24]:
weird1 = (auctions['end_date'] > auctions['death_year'])
weird2 = (auctions['start_date'] < auctions['birth_year'] + 10)

print("{} records where `end_date` > `death_year`".format(len(auctions[weird1])))
print("{} records where `start_date` < `birth_year + 10`".format(len(auctions[weird2])))

58 records where `end_date` > `death_year`
6044 records where `start_date` < `birth_year + 10`


For weirdness of the first type (`end_date > death_year`, ie artwork created after artist's death), I'll just replace artwork dates with `np.nan` since this just doesn't make sense to me.

In [25]:
auctions.loc[weird1, ['start_date', 'end_date']] = np.nan

However, the second situation seems like a bigger potential issue.

In [26]:
auctions[weird2]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
136,Pablo Picasso,Tête d'homme,1881-1973,oil on canvas,61.1 x 50 cm,"Dec 2, 2020",Christie's• New York,20th Century: Hong Kong to New York,Lot43,"US\$2,670,000","US\$1,500,000–US\$2,500,000(est)",NaN,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
137,Pablo Picasso,Femme nue assise,1881-1973,oil on board laid down on panel,49.5 x 36.2 cm,"Dec 2, 2020",Christie's• New York,20th Century: Hong Kong to New York,Lot48,NaN,"US\$1,200,000–US\$1,800,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
138,Pablo Picasso,Femme debout,1881-1973,oil on canvas,129 x 96.5 cm,"Dec 2, 2020",Christie's • New York,20th Century: Hong Kong to New York,Lot33,"US\$2,550,000","US\$3,000,000–US\$5,000,000 (est)",NaN,Spanish,Male,1881.0,1973.0,2020-12-02,1881.0,1973.0
140,Pablo Picasso,Nature morte aux tomates,1881-1973,oil on panel,92 x 73.5 cm,"Oct 22, 2020",Christie's • Paris,Paris Avant-garde,Lot13,"€968,000• US\$1,148,297","€900,000–€1,300,000(est)",NaN,Spanish,Male,1881.0,1973.0,2020-10-22,1881.0,1973.0
142,Pablo Picasso,La femme aux bas bleus,1881-1973,oil on board laid down on canvas,65 x 50 cm,"Oct 6, 2020",Christie's • New York,20th Century Evening Sale,Lot56,"US\$1,470,000","US\$1,000,000–US\$1,500,000 (est)",NaN,Spanish,Male,1881.0,1973.0,2020-10-06,1881.0,1973.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52971,Francis Bacon,Study of Portrait of John Edwards,1909-1992,oil on canvas,35.9 x 30.8 cm,"May 13, 2008","Christie's • New York, Rockefeller Center",Post War And Contemporary Art Evening Sale,Lot39,"US\$4,521,000","US\$4,000,000–US\$6,000,000 (est)",NaN,British,Male,1909.0,1992.0,2008-05-13,1909.0,1992.0
52972,Francis Bacon,Three Studies for Self-Portrait,1909-1992,"oil on canvas, in three parts",35.5 x 101 cm,"May 13, 2008","Christie's • New York, Rockefeller Center",Post War And Contemporary Art Evening Sale,Lot10,"US\$28,041,000","US\$25,000,000–US\$35,000,000 (est)",NaN,British,Male,1909.0,1992.0,2008-05-13,1909.0,1992.0
52974,Francis Bacon,Untitled (Head),1909-1992,oil on canvas,35.5 x 30.5cm,"Feb 6, 2008","Christie's • London, King Street",Post-War and Contemporary Art Evening Sale,Lot37,"£558,100 • US\$1,094,992","£500,000–£750,000 (est)",NaN,British,Male,1909.0,1992.0,2008-02-06,1909.0,1992.0
52981,Francis Bacon,Untitled (Figure on a Dais),1909-1992,huile sur toile,190 x 140 cm,"May 30, 2007",Christie's • Paris,Art d'Après-Guerre et Contemporain,Lot325,"€6,864,000 • US\$9,231,545","€4,000,000–€5,000,000 (est)",NaN,British,Male,1909.0,1992.0,2007-05-30,1909.0,1992.0


I can see immediately that there are some situations where the artwork date incorrectly has the artist's life span. Whenever this is the case (i.e., when `end_date == death_year and start_date == birth_year`) I'll assume the artwork dates are unknown.

In [27]:
mislabeled = (auctions['end_date'] == auctions['death_year']) & (auctions['start_date'] == auctions['birth_year'])

auctions.loc[mislabeled, ['start_date', 'end_date']] = np.nan

Now let's have another look at weirdness of the second variety.

In [28]:
auctions[(auctions['start_date'] < auctions['birth_year'] + 10)]

,artist_name,title,date,medium,dims,auction_date,auction_house,auction_sale,auction_lot,price_realized,estimate,bought_in,Nationality,Gender,birth_year,death_year,auction_date_parsed,start_date,end_date
342,Pablo Picasso,"Virgen de Guadalupe ,\n1848",1848,Oil on canvas,57.0 x 88.0 cm,"Sep 12, 2017",Morton Casa de Subastas • Lomas de Chapultepec,Gran Sabasta de Antiguedades,Lot138,NaN,"MX\$40,000–MX\$60,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2017-09-12,1848.0,1848.0
343,Pablo Picasso,"Divino rostro ,\n1839",1839,Oil on canvas,43.0 x 56.0 cm,"Sep 12, 2017",Morton Casa de Subastas • Lomas de Chapultepec,Gran Sabasta de Antiguedades,Lot130,NaN,"MX\$10,000–MX\$15,000 (est)",1.0,Spanish,Male,1881.0,1973.0,2017-09-12,1839.0,1839.0
1297,Jean-Michel Basquiat,Untitled,1960,"graphite, colored pencil, oil crayon and acryl...",104.14 x 75.25 cm,"Nov 17, 2020",Sotheby's,Contemporary Art Day Auction,Lot133,"US\$1,000,100","US\$900,000–US\$1,200,000(est)",NaN,American,Male,1960.0,1988.0,2020-11-17,1960.0,1960.0
1298,Jean-Michel Basquiat,BLACK,1960,"acrylic, oil, graphite, crayon and Xerox on wood",92.7 x 126.4 x 29.2 cm,"Oct 28, 2020",Sotheby's,Contemporary Art Evening Auction,Lot18,"US\$8,134,650","US\$4,000,000–US\$6,000,000 (est)",NaN,American,Male,1960.0,1988.0,2020-10-28,1960.0,1960.0
1453,Jean-Michel Basquiat,Collaboration,1928-1987,"colored oilsticks, synthetic polymer paint and...",51.2 x 40.7 cm,"May 14, 2014","Christie's • New York, Rockefeller Center",Post-War and Contemporary Afternoon Session,Lot479,"US\$581,000","US\$300,000–US\$500,000 (est)",NaN,American,Male,1960.0,1988.0,2014-05-14,1928.0,1987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52453,Robert Indiana,SMALL DIAMOND HEXAGON 6,1928,oil on canvas,86.4 x 86.4 cm,"Oct 2, 2020",Sotheby's,Contemporary Curated,Lot41,NaN,"US\$150,000–US\$250,000(est)",1.0,American,Male,1928.0,2018.0,2020-10-02,1928.0,1928.0
52580,Robert Indiana,CLASSIC LOVE,1820,chrome-dyed hand carved tufted archival New Ze...,1820 x 1820 mm,"Sep 16, 2008","Christie's • London, South Kensington",Interiors - 20th Century Edition including Mod...,Lot172,"£1,625 • US\$2,917","£1,500–£2,000(est)",NaN,American,Male,1928.0,2018.0,2008-09-16,1820.0,1820.0
52581,Robert Indiana,CHOSEN LOVE,1820,chrome-dyed hand carved tufted archival New Ze...,1820 x 1820 mm,"Sep 16, 2008","Christie's • London, South Kensington",Interiors - 20th Century Edition including Mod...,Lot106,"£1,375 • US\$2,468","£1,000–£1,500(est)",NaN,American,Male,1928.0,2018.0,2008-09-16,1820.0,1820.0
52725,Leonora Carrington,Mi general aspirina y sus hombres,1917,oil on canvas,76.2 x 101.3 cm,"Nov 17, 2010","Christie's • New York, Rockefeller Center",Latin American Sale,Lot54,"US\$206,500","US\$150,000–US\$200,000(est)",NaN,British,Female,1917.0,2011.0,2010-11-17,1917.0,1917.0


This is much more reasonable. It's not that many records that are problematic, so I'll just assume the dates for these works are unknown and replace them with `np.nan`.

In [29]:
mislabeled = (auctions['start_date'] < auctions['birth_year'] + 10)

auctions.loc[mislabeled, ['start_date', 'end_date']] = np.nan

## 7. `dimensions`
### 7.1 Exploration
Let's see how varied this features is.

In [30]:
# Create masks for expected formats in cm, mm, or in units
expected_cm_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?cm.*?').fillna(False))
expected_in_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?in.*?').fillna(False))
expected_mm_format = (auctions['dims'].str.match(r'.*?\d+?\.?\d*? ?(x|by) ?\d+?\.?\d*? ?mm.*?').fillna(False))

auctions.loc[~expected_cm_format & ~expected_in_format & ~expected_mm_format, 'dims'].dropna()

59                400 Diameter 390mm.
83                diameter 250mm 10in
113             width 190 mm 7 1/2 in
158                           19.5 cm
162                           19.0 cm
                     ...             
50370             45 7/8 x 34 7/8 in.
50435                    cm 40,5x32,5
51587    8⅛ by 24 in.\n20.6 by 61 cm.
51944                           6 cm.
52423                   2 by 1 5/8 in
Name: dims, Length: 723, dtype: object

In [31]:
# Ensure that each value has a unit of measurement
auctions.loc[~auctions['dims'].str.contains('in|cm|mm|CM', regex=True).fillna(False), 'dims'].dropna()

25316    dimensions variable
Name: dims, dtype: object

Because each value (except one) has a unit of measurment, I'll use that as a boundary, extracting the portion of this field specific to each unit of measurement, and then parse from there. Doing it this way, however, will remove text specifying whether a measurement is a diameter or not--that's information I want, too. For now, I'll create a flag for works that have diameter measurements, since this will require a different calculation for area.

### 7.2 Preliminary Extraction

In [32]:
# Extract cm dimensions
auctions['dims_cm'] = auctions['dims'].str.lower().str.extract(r'(\d[^(in|mm)]+ ?cm)')

# Extract in. dimensions
auctions['dims_in'] = auctions['dims'].str.lower().str.extract(r'(\d[^(cm|mm)]+ ?in)')

# Extract mm dimensions
auctions['dims_mm'] = auctions['dims'].str.lower().str.extract(r'(\d[^(in|cm)]+ ?mm)')

# Create flag for diameter
auctions['is_diameter'] = auctions['dims'].str.lower().str.match(r'.*diameter.*').fillna(False)

auctions[['dims', 'dims_cm', 'dims_in', 'dims_mm']].sample(10)

,dims,dims_cm,dims_in,dims_mm
38539,121.92 x 182.88 in,NaN,121.92 x 182.88 in,NaN
38329,58 ¾ x 58 ¼ in. (149.2 x 148 cm.),149.2 x 148 cm,58 ¾ x 58 ¼ in,NaN
10535,45.7 x 50.8 cm,45.7 x 50.8 cm,NaN,NaN
45696,78.74 x 55.88 in,NaN,78.74 x 55.88 in,NaN
8416,22.86 x 15.24 in,NaN,22.86 x 15.24 in,NaN
43994,90.0 x 120.5 cm,90.0 x 120.5 cm,NaN,NaN
14388,55 x 50 cm,55 x 50 cm,NaN,NaN
21627,63.8 x 97.0 in,NaN,63.8 x 97.0 in,NaN
20607,15.2 x 15.2 cm,15.2 x 15.2 cm,NaN,NaN
10197,24.2 x 27.2 cm,24.2 x 27.2 cm,NaN,NaN


This roughly looks like what's expected but let's check out the edge cases.

In [33]:
# Review extractions where `dims` is not in an expected format
edge_cases = (
    ~expected_cm_format & 
    ~expected_in_format & 
    ~expected_mm_format & 
    auctions['dims'].notna()
)

auctions.loc[edge_cases, ['dims', 'dims_cm', 'dims_in', 'dims_mm']]

,dims,dims_cm,dims_in,dims_mm
59,400 Diameter 390mm.,NaN,NaN,390mm
83,diameter 250mm 10in,NaN,10in,250mm
113,width 190 mm 7 1/2 in,NaN,7 1/2 in,190 mm
158,19.5 cm,19.5 cm,NaN,NaN
162,19.0 cm,19.0 cm,NaN,NaN
...,...,...,...,...
50370,45 7/8 x 34 7/8 in.,NaN,45 7/8 x 34 7/8 in,NaN
50435,"cm 40,5x32,5",NaN,NaN,NaN
51587,8⅛ by 24 in.\n20.6 by 61 cm.,20.6 by 61 cm,8⅛ by 24 in,NaN
51944,6 cm.,6 cm,NaN,NaN


This is looking quite good to me so far. Before continuing, I just want to ensure that there are no remaining `dims` values that were not extracted to `dims_cm`, `dims_in`, or `dims_mm`.

In [34]:
# Ensure all `dims` values were extracted in one form or another
cols = ['dims', 'dims_cm', 'dims_in', 'dims_mm']

mask = (
    auctions['dims'].notna() &
    auctions['dims_cm'].isna() &
    auctions['dims_mm'].isna() &
    auctions['dims_in'].isna()
)

auctions.loc[mask, cols]

,dims,dims_cm,dims_in,dims_mm
1335,cm 61x51,NaN,NaN,NaN
15749,"cm 41x35,5",NaN,NaN,NaN
15767,"cm 60x75,2",NaN,NaN,NaN
15797,"cm 27,5x24",NaN,NaN,NaN
15813,"cm 45,5x38",NaN,NaN,NaN
15814,"cm 24,4x31,3",NaN,NaN,NaN
16396,"cm 25,4x30,3",NaN,NaN,NaN
23191,"cm 50,5x40",NaN,NaN,NaN
25316,dimensions variable,NaN,NaN,NaN
43312,"cm 45,7x54,5",NaN,NaN,NaN


Not quite, so I'll need to extract these as well (i.e., dims prepended by 'cm')

In [35]:
auctions.loc[mask, 'dims_cm'] = (
    auctions.loc[mask, 'dims'].str.replace(',', '.')
    .str.extract(r'^cm ?(\d+\.?\d*x\d+\.?\d*)')[0]
)
auctions.loc[mask, cols]

,dims,dims_cm,dims_in,dims_mm
1335,cm 61x51,61x51,NaN,NaN
15749,"cm 41x35,5",41x35.5,NaN,NaN
15767,"cm 60x75,2",60x75.2,NaN,NaN
15797,"cm 27,5x24",27.5x24,NaN,NaN
15813,"cm 45,5x38",45.5x38,NaN,NaN
15814,"cm 24,4x31,3",24.4x31.3,NaN,NaN
16396,"cm 25,4x30,3",25.4x30.3,NaN,NaN
23191,"cm 50,5x40",50.5x40,NaN,NaN
25316,dimensions variable,NaN,NaN,NaN
43312,"cm 45,7x54,5",45.7x54.5,NaN,NaN


### 7.3 Secondary Extraction
Next, I want to extract width, height, and depth for each of these.

In [36]:
# Extract `width` dim from `dims_cm`
auctions['width_cm'] = auctions['dims_cm'].str.extract(r'^(\d+.*?)[^\d\. ].*?$')

# Extract `height` dim from `dims_cm`
auctions['height_cm'] = auctions['dims_cm'].str.extract(r'^\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

# Extract `depth` dim from `dims_cm`
auctions['depth_cm'] = auctions['dims_cm'].str.extract(r'^\d+.*?[^\d\.,].*?\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

auctions.loc[auctions['dims_cm'].notna(), ['dims_cm', 'width_cm', 'height_cm', 'depth_cm']].sample(20)

,dims_cm,width_cm,height_cm,depth_cm
17765,92 by 65.5 cm,92,65.5,NaN
3392,50.8 x 60.4 cm,50.8,60.4,NaN
532,99.4 x 80.8 cm,99.4,80.8,NaN
34509,64.5 x 54.5 cm,64.5,54.5,NaN
829,115.8 x 89.2 cm,115.8,89.2,NaN
3323,35.5 by 28 cm,35.5,28,NaN
44480,18 x 28.5 cm,18,28.5,NaN
45348,304.8 x 322.6 cm,304.8,322.6,NaN
27468,200 by 150 cm,200,150,NaN
11779,66.04 x 81.28 cm,66.04,81.28,NaN


This looks good to me, so I'll do the same for the other units of measurement.

In [37]:
# Extract `width` dim from `dims_mm`
auctions['width_mm'] = auctions['dims_mm'].str.extract(r'^(\d+.*?)[^\d\. ].*?$')

# Extract `height` dim from `dims_mm`
auctions['height_mm'] = auctions['dims_mm'].str.extract(r'^\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

# Extract `depth` dim from `dims_mm`
auctions['depth_mm'] = auctions['dims_mm'].str.extract(r'^\d+.*?[^\d\.,].*?\d+.*?[^\d\.,].*?(\d+.*?)[^\d\.,].*?$')

auctions.loc[auctions['dims_mm'].notna(), ['dims_mm', 'width_mm', 'height_mm', 'depth_mm']].sample(20)

,dims_mm,width_mm,height_mm,depth_mm
252,260mm,260,NaN,NaN
16261,305 x 305 x 40 mm,305,305,40
133,215 by 215 by 20 mm,215,215,20
59,390mm,390,NaN,NaN
52580,1820 x 1820 mm,1820,1820,NaN
11487,282 x 440mm,282,440,NaN
121,215 by 215 by 20 mm,215,215,20
23,190 by 189mm,190,189,NaN
39385,1200 x 1200 x 50 mm,1200,1200,50
254,160mm,160,NaN,NaN


Inches is a little trickier since I've noticed some numbers use fractions, so I have to account for this.

In [38]:
# Extract `width` dim from `dims_in`
auctions['width_in'] = auctions['dims_in'].str.extract(r'^(\d+.*?)[^\d\. /].*?$')

# Extract `height` dim from `dims_in`
auctions['height_in'] = auctions['dims_in'].str.extract(r'^\d+.*?[^\d\., /].*?(\d+.*?)[^\d\., /].*?$')

# Extract `depth` dim from `dims_in`
auctions['depth_in'] = auctions['dims_in'].str.extract(r'^\d+.*?[^\d\., /].*?\d+.*?[^\d\., /].*?(\d+.*?)[^\d\., /].*?$')

auctions.loc[auctions['dims_in'].notna(), ['dims_in', 'width_in', 'height_in', 'depth_in']].sample(20)

,dims_in,width_in,height_in,depth_in
41602,81.51 x 65.99 in,81.51,65.99,NaN
35895,81.28 x 99.06 in,81.28,99.06,NaN
21806,65.0 x 50.01 in,65.0,50.01,NaN
25982,40.64 x 33.02 in,40.64,33.02,NaN
49346,35.56 x 48.26 in,35.56,48.26,NaN
25318,36 x 48 in,36,48,NaN
2850,101.6 x 101.6 in,101.6,101.6,NaN
15689,29.49 x 29.49 in,29.49,29.49,NaN
15954,15.24 x 10.16 in,15.24,10.16,NaN
37473,32.99 x 41.0 in,32.99,41.0,NaN


In [39]:
# Create function to parse fractions
import re

def parse_measurement(measurement):
      
    match = re.match(r'(\d+) (\d+)\/(\d+)', measurement)
    
    if match:
        num = int(match.group(1)) + int(match.group(2)) / int(match.group(3))
    else:
        num = float(re.match(r'(\d+(\.\d+)?)', measurement).group(1))
    
    return num

In [40]:
# Create masks
has_width_in = auctions['width_in'].notna()
has_height_in = auctions['height_in'].notna()
has_depth_in = auctions['depth_in'].notna()

# Parse fractions
auctions.loc[has_width_in, 'width_in'] = auctions.loc[has_width_in, 'width_in'].apply(parse_measurement)
auctions.loc[has_height_in, 'height_in'] = auctions.loc[has_height_in, 'height_in'].apply(parse_measurement)
auctions.loc[auctions['depth_in'].notna(), 'depth_in'] = auctions.loc[auctions['depth_in'].notna(), 'depth_in'].apply(parse_measurement)

cols = ['dims_in', 'width_in', 'height_in', 'depth_in']

auctions.loc[auctions['dims_in'].str.contains('/').fillna(False), cols]

,dims_in,width_in,height_in,depth_in
6,8 3/4 x 10 1/2 in,8.75,10.5,NaN
7,15 x 21 3/4 in,15.0,21.75,NaN
8,36 3/8 x 14 3/8 in,36.375,14.375,NaN
33,7 1/4 x 9 3/8 in,7.25,9.375,NaN
66,4 ¾ x 7 1/8 in,4.0,7.125,NaN
...,...,...,...,...
52433,24 x 24 1/8 in,24.0,24.125,NaN
52816,82 1/2 x 58 1/8 in,82.5,58.125,NaN
52877,25 3/4 x 21 7/8 in,25.75,21.875,NaN
52879,57 1/4 x 43 in,57.25,43.0,NaN


### 7.4 Convert Measurment Features to Float

In [41]:
# cm
print("`width_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_cm'].dropna(), errors='coerce').isna().sum()))
print("`height_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_cm'].dropna(), errors='coerce').isna().sum()))
print("`depth_cm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_cm'].dropna(), errors='coerce').isna().sum()))

# in
print("`width_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_in'].dropna(), errors='coerce').isna().sum()))
print("`height_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_in'].dropna(), errors='coerce').isna().sum()))
print("`depth_in`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_in'].dropna(), errors='coerce').isna().sum()))

# mm
print("`width_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['width_mm'].dropna(), errors='coerce').isna().sum()))
print("`height_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['height_mm'].dropna(), errors='coerce').isna().sum()))
print("`depth_mm`: {} non-numeric values.".format(
    pd.to_numeric(auctions['depth_mm'].dropna(), errors='coerce').isna().sum()))

`width_cm`: 5 non-numeric values.
`height_cm`: 16 non-numeric values.
`depth_cm`: 0 non-numeric values.
`width_in`: 0 non-numeric values.
`height_in`: 0 non-numeric values.
`depth_in`: 0 non-numeric values.
`width_mm`: 0 non-numeric values.
`height_mm`: 0 non-numeric values.
`depth_mm`: 0 non-numeric values.


I'll coerce the non-numeric `cm` values to NaN and then conver all features to floats.

In [42]:
auctions['width_cm'] = pd.to_numeric(auctions['width_cm'], errors='coerce').astype('float')
auctions['height_cm'] = pd.to_numeric(auctions['height_cm'], errors='coerce').astype('float')
auctions['depth_cm'] = auctions['depth_cm'].astype('float')

auctions['width_in'] = auctions['width_in'].astype('float')
auctions['height_in'] = auctions['height_in'].astype('float')
auctions['depth_in'] = auctions['depth_in'].astype('float')

auctions['width_mm'] = auctions['width_mm'].astype('float')
auctions['height_mm'] = auctions['height_mm'].astype('float')
auctions['depth_mm'] = auctions['depth_mm'].astype('float')

### 7.5 Standardizing Units of Measurement to cm
Last let's see where we need to convert either mm or in to cm (scenarios where `dims_cm` is null but either `dims_in` or `dims_mm` is not).

First, the easy task: converting mm to cm

In [43]:
# Review situations where we have mm but not cm
cols = ['dims', 'dims_cm', 'dims_mm', 'dims_in']

have_cm_dims = (auctions['dims_cm'].notna())
have_mm_dims = (auctions['dims_mm'].notna())
have_in_dims = (auctions['dims_in'].notna())

auctions.loc[~have_cm_dims & have_mm_dims, cols]

,dims,dims_cm,dims_mm,dims_in
22,185 by 184mm\n7¼ by 7¼in,NaN,185 by 184mm,7¼ by 7¼in
23,190 by 189mm\n7½ by 7½in,NaN,190 by 189mm,7½ by 7½in
59,400 Diameter 390mm.,NaN,390mm,NaN
79,314 by 315mm\n12⅜ by 12 3/8in,NaN,314 by 315mm,12⅜ by 12 3/8in
80,170 by 170mm\n6¾ by 6¾in,NaN,170 by 170mm,6¾ by 6¾in
81,314 by 315mm\n12⅜ by 12 3/8in,NaN,314 by 315mm,12⅜ by 12 3/8in
82,310 by 310mm\n12⅛ by 12 1/8in,NaN,310 by 310mm,12⅛ by 12 1/8in
83,diameter 250mm 10in,NaN,250mm,10in
113,width 190 mm 7 1/2 in,NaN,190 mm,7 1/2 in
114,310 by 310 by 20 mm\n12 by 12 by 3/4 in\n515 b...,NaN,310 by 310 by 20 mm,12 by 12 by 3/4 in


In [44]:
# Convert mm to cm when we have mm but not cm
mm_dims = ['width_mm', 'height_mm', 'depth_mm']
cm_dims = ['width_cm', 'height_cm', 'depth_cm']

for dims in zip(cm_dims, mm_dims):
    auctions.loc[~have_cm_dims & have_mm_dims, dims[0]] = auctions.loc[~have_cm_dims & have_mm_dims, dims[1]]/ 10

auctions.loc[~have_cm_dims & have_mm_dims, cm_dims + mm_dims]

,width_cm,height_cm,depth_cm,width_mm,height_mm,depth_mm
22,18.5,18.4,NaN,185.0,184.0,NaN
23,19.0,18.9,NaN,190.0,189.0,NaN
59,39.0,NaN,NaN,390.0,NaN,NaN
79,31.4,31.5,NaN,314.0,315.0,NaN
80,17.0,17.0,NaN,170.0,170.0,NaN
81,31.4,31.5,NaN,314.0,315.0,NaN
82,31.0,31.0,NaN,310.0,310.0,NaN
83,25.0,NaN,NaN,250.0,NaN,NaN
113,19.0,NaN,NaN,190.0,NaN,NaN
114,31.0,31.0,2.0,310.0,310.0,20.0


Now we can convert inches dims to cm dims where we're still missing the latter.

In [45]:
# Review situations where we have in. but not cm or mm
cols = ['dims', 'dims_cm', 'dims_mm', 'dims_in']

have_cm_dims = (auctions['dims_cm'].notna())
have_mm_dims = (auctions['dims_mm'].notna())
have_in_dims = (auctions['dims_in'].notna())

auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, cols]

,dims,dims_cm,dims_mm,dims_in
24,53⁄4 x 57⁄8 in. (14.5 x 14.8 in.),NaN,NaN,53⁄4 x 57⁄8 in
270,19 5/8 x 28 5/8 in.,NaN,NaN,19 5/8 x 28 5/8 in
274,15 by 15 in,NaN,NaN,15 by 15 in
416,49.83 x 65.07 in,NaN,NaN,49.83 x 65.07 in
417,19.0 x 14.3 in,NaN,NaN,19.0 x 14.3 in
...,...,...,...,...
53029,144.78 x 198.12 in,NaN,NaN,144.78 x 198.12 in
53030,68.58 x 83.82 in,NaN,NaN,68.58 x 83.82 in
53031,147.32 x 198.12 in,NaN,NaN,147.32 x 198.12 in
53032,116.84 x 152.4 in,NaN,NaN,116.84 x 152.4 in


In [46]:
# Convert in. to cm when we have in. but not cm or mm
in_dims = ['width_in', 'height_in', 'depth_in']
cm_dims = ['width_cm', 'height_cm', 'depth_cm']

for dims in zip(cm_dims, in_dims):
    auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, dims[0]] = auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, dims[1]] * 2.54

auctions.loc[~have_cm_dims & ~have_mm_dims & have_in_dims, cm_dims + in_dims]

,width_cm,height_cm,depth_cm,width_in,height_in,depth_in
24,134.6200,10.1600,144.78,53.000,4.000,57.0
270,49.8475,72.7075,NaN,19.625,28.625,NaN
274,38.1000,38.1000,NaN,15.000,15.000,NaN
416,126.5682,165.2778,NaN,49.830,65.070,NaN
417,48.2600,36.3220,NaN,19.000,14.300,NaN
...,...,...,...,...,...,...
53029,367.7412,503.2248,NaN,144.780,198.120,NaN
53030,174.1932,212.9028,NaN,68.580,83.820,NaN
53031,374.1928,503.2248,NaN,147.320,198.120,NaN
53032,296.7736,387.0960,NaN,116.840,152.400,NaN


## 8. `title`, `medium`, `auction_sale`, `auction_lot`
This I'm going to ignore since I don't see it being a meaningful feature in my model. Perhaps down the road I can incorporate NLP to make use of features like `title`, `medium`, and `auction_sale`

## 9 `auction_house`

In [47]:
# Review `auction_house` feature
auctions.loc[auctions['auction_house'].str.contains('•'), 'auction_house'].value_counts()

Christie's • New York                           7218
Christie's • London                             3471
Christie's • New York, Rockefeller Center       3026
Christie's • London, King Street                1910
Christie's• New York                            1424
                                                ... 
Matsa• Ramat Gan                                   1
Wolf's • Cleveland OH                              1
Charlton Hall Galleries • Columbia SC              1
Museumsbygningen Kunstauktioner • Copenhagen       1
Selkirk's • Saint Louis                            1
Name: auction_house, Length: 1313, dtype: int64

In [48]:
# Extract location information where available
auctions['auction_house_loc'] = auctions['auction_house'].str.extract(r'• ?([^,]*)')

auctions.loc[auctions['auction_house_loc'].notna(), ['auction_house', 'auction_house_loc']].sample(10)

,auction_house,auction_house_loc
20813,Christie's• London,London
52885,Sotheby's • London,London
35350,"Christie's • London, King Street",London
20306,Ansorena • Madrid,Madrid
14610,Christie's • London,London
20555,Christie's • New York,New York
16685,Christie's• New York,New York
32733,Phillips • Hong Kong,Hong Kong
45844,"Rieunier • Bailly-Pommery, Mathias, Paris",Bailly-Pommery
14588,"Christie's • London, King Street",London


In [49]:
auctions['auction_house_loc'].value_counts().head(20)

New York                14639
London                   8308
Paris                    4713
Hong Kong                2149
Cologne                   644
Munich                    483
Beijing                   472
Poulain                   403
Zurich                    400
Berlin                    392
Milan                     372
Vienna                    367
Bern                      345
Tokyo                     311
South Kensington          271
Royere et Lajeunesse      262
Prato                     254
Amsterdam                 247
Taipei                    246
East                      238
Name: auction_house_loc, dtype: int64

In [50]:
# Extract auction house name information
auctions['auction_house_name'] = auctions['auction_house'].str.extract(r'^([^•]*)')[0].str.strip()

auctions.loc[auctions['auction_house_loc'].isna(), ['auction_house', 'auction_house_name']].sample(10)

,auction_house,auction_house_name
24038,Phillips,Phillips
9826,Phillips,Phillips
39182,Phillips,Phillips
34341,Sotheby's,Sotheby's
46008,Sotheby's,Sotheby's
15723,Sotheby's,Sotheby's
8826,Christie's,Christie's
28525,Christie's,Christie's
52446,Sotheby's,Sotheby's
39575,Christie's,Christie's


In [51]:
# Standardize `phillips` auction house name
phillips = (auctions['auction_house_name'].str.startswith('Phillips'))

auctions.loc[phillips, 'auction_house_name'] = 'Phillips'

auctions['auction_house_name'].value_counts().head(20)

Christie's                      24605
Sotheby's                        9230
Phillips                         3935
Artcurial                         781
Bonhams                           773
Cornette de Saint Cyr             504
Ketterer Kunst                    462
Tajan                             405
Kunsthaus Lempertz                385
Dorotheum                         374
Poly International Auction        363
Ravenel                           334
Galerie Kornfeld                  310
Guy Loudmer                       300
Francis Briest                    276
Villa Grisebach Auktionen         269
Koller Auktionen                  265
Versailles Encheres - Perrin      262
Farsetti Arte                     250
China Guardian                    215
Name: auction_house_name, dtype: int64

## 10. `price_realized`

In [52]:
# Extract USD price from `price_realized`
auctions['price_realized_USD'] = auctions['price_realized'].str.replace(',','').str.extract(r'US\\\$(\d+)')

auctions[['price_realized', 'price_realized_USD']].sample(20)

,price_realized,price_realized_USD
15080,"€638,726 • US\$888,563",888563
11111,"US\$6,500,000",6500000
15237,"€270,000 • US\$362,771",362771
44778,NaN,NaN
36549,"US\$460,000",460000
52030,"US\$262,400",262400
38847,NaN,NaN
2839,"€105,000 • US\$116,091",116091
817,NaN,NaN
37256,"£289,250 • US\$455,652",455652


## 11. `estimate_range`
I'm going to ignore this for the time being since the idea would be to make a prediction without the benefit of the estimate.

## 12. Engineer Features
- artwork area (cm sq): `width_cm` x `height_cm` or pi x (`width_cm` / 2) ^ 2 for circular works
- artwork volume (cm cu): `width_cm` x `height_cm` x `depth_cm` (shouldn't come up since we're dealing with 2D paintings, but there are a number of works with depth measurements regardless)
- living: was the artist living at the time of auction?
- years after death: years after artist death that the work was auctioned (if artist was deceased)
- age_at_auction: artist age at time of auction (if artist was alive)
- age_at_artwork_completion: artist age when artwork completed (proxy for early-, mid-, and late-career)
- artwork age: artwork age at time of auction
- years ago: years before 2023 that auction was held

### 12.1 `area_cm_sq` (no diameter)

In [53]:
# Compute area for standard works (no diameter measurement)
has_diameter = (auctions['is_diameter'] == 1)

auctions.loc[~has_diameter, 'area_cm_sq'] = (
    auctions.loc[~has_diameter, 'width_cm'] * auctions.loc[~has_diameter, 'height_cm']
)

cols = ['dims_cm', 'is_diameter', 'width_cm', 'height_cm', 'depth_cm', 'area_cm_sq']

auctions[cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq
35266,NaN,False,330.200,226.0600,NaN,74645.012000
17935,NaN,False,225.806,367.7412,NaN,83038.169407
37181,NaN,False,467.360,236.1946,NaN,110387.908256
21834,117 by 89 cm,False,117.000,89.0000,NaN,10413.000000
41697,92 x 73 cm,False,92.000,73.0000,NaN,6716.000000
749,27 x 22 cm,False,27.000,22.0000,NaN,594.000000
4874,25.4 x 30.5 cm,False,25.400,30.5000,NaN,774.700000
25929,60.96 x 50.8 cm,False,60.960,50.8000,NaN,3096.768000
38812,160 x 200 cm,False,160.000,200.0000,NaN,32000.000000
6684,NaN,False,68.580,55.3466,NaN,3795.669828


### 12.2 `area_cm_sq`  (with diamter)

In [54]:
# Compute area for non-standard works (with diameter measurement)
auctions.loc[has_diameter, 'area_cm_sq'] = (
    3.1415926 * (auctions.loc[has_diameter, 'width_cm'] / 2) ** 2
)

auctions.loc[has_diameter, cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq
29106,213.4 cm,True,213.40,NaN,NaN,35766.686176
29221,365.8 cm,True,365.80,NaN,NaN,105093.843708
23950,50.8 cm,True,50.80,NaN,NaN,2026.829882
83,NaN,True,25.00,NaN,NaN,490.873844
23839,51.3 cm,True,51.30,NaN,NaN,2066.924457
24086,NaN,True,182.88,NaN,NaN,26267.715268
29169,52 cm,True,52.00,NaN,NaN,2123.716598
39350,100cm,True,100.00,NaN,NaN,7853.981500
14641,220cm,True,220.00,NaN,NaN,38013.270460
29109,152.4 cm,True,152.40,NaN,NaN,18241.468936


### 12.3 `volume_cm_cu` 

In [55]:
# Compute volume for works with depth measurement
auctions['volume_cm_cu'] = (
    auctions['width_cm'] * auctions['height_cm'] * auctions['depth_cm']
)

cols = ['dims_cm', 'is_diameter', 'width_cm', 'height_cm', 'depth_cm', 'area_cm_sq', 'volume_cm_cu']

auctions.loc[auctions['depth_cm'].notna(), cols].sample(10)

,dims_cm,is_diameter,width_cm,height_cm,depth_cm,area_cm_sq,volume_cm_cu
24961,287 x 261.6 x 111.7 cm,False,287.0000,261.6000,111.70,75079.200000,8.386347e+06
46903,37.5 x 38 x 17.5 cm,False,37.5000,38.0000,17.50,1425.000000,2.493750e+04
29917,190.5 x 190.5 x 10.8cm,False,190.5000,190.5000,10.80,36290.250000,3.919347e+05
3906,10.1 x 6.9 x 6.9 cm,False,10.1000,6.9000,6.90,69.690000,4.808610e+02
24897,246.7 x 205.7 x 20.3 cm,False,246.7000,205.7000,20.30,50746.190000,1.030148e+06
46217,214 x 155 x 13.5 cm,False,214.0000,155.0000,13.50,33170.000000,4.477950e+05
48122,31.6 x 39.4 x 2.9 cm,False,31.6000,39.4000,2.90,1245.040000,3.610616e+03
49605,20.9 x 15.8 x 1.2 cm,False,20.9000,15.8000,1.20,330.220000,3.962640e+02
25420,NaN,False,83.7946,71.0946,16.51,5957.343569,9.835574e+04
39161,67 x 62 x 46 cm,False,67.0000,62.0000,46.00,4154.000000,1.910840e+05


### 12.4 `living`

In [56]:
# Create mask for if artist is still living or was living at time of auction
mask = (
    (auctions['auction_date_parsed'].dt.year <= auctions['death_year']) # Living at time of auction
    | auctions['death_year'].isna() # Still living
)

auctions['living'] = np.where(mask, 1, 0)

cols = ['auction_date_parsed', 'death_year', 'living']

auctions[cols].sample(20)

,auction_date_parsed,death_year,living
35641,2001-06-20,1999.0,0
50144,2001-06-25,2019.0,1
44050,2016-05-11,1991.0,0
2971,2014-11-27,1987.0,0
5472,1994-05-03,1987.0,0
45776,2000-06-23,1963.0,0
27139,2011-05-09,2021.0,1
15746,2012-02-14,1968.0,0
35007,2011-09-30,1999.0,0
37931,2002-04-23,1994.0,0


### 12.5 `years_after_death_of_auction`

In [57]:
# Compute years after artist's death that auction was held for artists no longer living at time of auction
deceased = (auctions['living'] == 0)

auctions.loc[deceased, 'years_after_death_of_auction'] = (
    auctions.loc[deceased, 'auction_date_parsed'].dt.year - auctions.loc[deceased, 'death_year']
)

cols = ['auction_date_parsed', 'death_year', 'years_after_death_of_auction']

auctions[cols].sample(10)

,auction_date_parsed,death_year,years_after_death_of_auction
16778,2018-11-14,1992.0,26.0
23758,1992-11-19,NaN,NaN
50490,2008-02-22,1987.0,21.0
18285,2007-05-10,1944.0,63.0
51917,2011-05-25,NaN,NaN
38581,1993-11-10,2011.0,NaN
28893,2015-10-17,NaN,NaN
17092,2021-03-25,1954.0,67.0
18237,2012-11-08,1944.0,68.0
44708,2006-05-10,2004.0,2.0


### 12.6 `artist_age_at_auction`

In [58]:
# Compute artist's age at auction if artist living at time of auction

auctions.loc[~deceased, 'artist_age_at_auction'] = (
    auctions.loc[~deceased, 'auction_date_parsed'].dt.year - auctions.loc[~deceased, 'birth_year']
)

cols = ['auction_date_parsed', 'birth_year', 'death_year', 'artist_age_at_auction']

auctions[cols].sample(10)

,auction_date_parsed,birth_year,death_year,artist_age_at_auction
187,2020-02-05,1881.0,1973.0,NaN
41577,2016-05-09,1879.0,1953.0,NaN
26999,2021-05-13,1920.0,2021.0,101.0
26333,2007-06-18,1830.0,1903.0,NaN
1615,2008-05-15,1960.0,1988.0,NaN
36514,2008-05-15,1954.0,NaN,54.0
19909,2011-06-21,1841.0,1919.0,NaN
36676,2021-04-14,1923.0,1994.0,NaN
13978,2011-06-23,1893.0,1983.0,NaN
44071,2014-05-14,1915.0,1991.0,NaN


### 12.7 `artist_age_at_artwork_completeion`

In [59]:
# Compute artist age at time of artwork completion
auctions['artist_age_at_artwork_completion'] = auctions['end_date'] - auctions['birth_year']

cols = ['end_date', 'birth_year', 'artist_age_at_artwork_completion']

auctions[cols].sample(10)

,end_date,birth_year,artist_age_at_artwork_completion
50866,NaN,1904.0,NaN
18572,1973.0,1937.0,36.0
10486,NaN,1923.0,NaN
13778,NaN,1893.0,NaN
33204,1954.0,1914.0,40.0
40126,NaN,1867.0,NaN
28324,NaN,1941.0,NaN
1107,1941.0,1881.0,60.0
10927,NaN,1898.0,NaN
48187,1988.0,1958.0,30.0


### 12.8 `artwork_age_at_auction`

In [60]:
# Compute artwork age at time of auction
auctions['artwork_age_at_auction'] = auctions['auction_date_parsed'].dt.year - auctions['end_date']

cols = ['auction_date_parsed', 'end_date', 'artwork_age_at_auction']

auctions[cols].sample(10)

,auction_date_parsed,end_date,artwork_age_at_auction
1357,2017-12-06,NaN,NaN
9647,2018-04-01,NaN,NaN
20271,2007-06-19,NaN,NaN
10242,2009-06-17,2005.0,4.0
18043,1993-05-12,1950.0,43.0
27239,2021-06-08,2013.0,8.0
26047,2020-02-05,1876.0,144.0
2916,2015-05-12,1984.0,31.0
6747,2013-05-26,NaN,NaN
13121,1989-11-15,NaN,NaN


### 12.8 `years_ago_of_auction`

In [61]:
# Compute how many years from 2023 auction was held
auctions['years_ago_of_auction'] = 2023 - auctions['auction_date_parsed'].dt.year

cols = ['auction_date_parsed', 'years_ago_of_auction']

auctions[cols].sample(10)

,auction_date_parsed,years_ago_of_auction
37175,2013-01-09,10
50837,2021-11-11,2
41889,1997-11-12,26
42709,1997-05-14,26
10190,2010-09-22,13
12362,2014-10-24,9
19674,2013-10-21,10
19525,2015-05-15,8
1994,1988-11-10,35
28890,2016-02-10,7


## 13 Re-order columns and Export CSV

In [62]:
new_col_order = [
    'artist_name', 'Nationality', 'Gender', 'birth_year', 'death_year',
    'title', 'date', 'start_date', 'end_date', 'medium', 
    'dims', 'dims_cm', 'dims_in', 'dims_mm', 'is_diameter', 
    'width_cm', 'height_cm', 'depth_cm', 
    'width_mm', 'height_mm', 'depth_mm', 
    'width_in', 'height_in', 'depth_in', 
    'area_cm_sq', 'volume_cm_cu', 
    'auction_date', 'auction_date_parsed', 'auction_house', 'auction_house_loc', 'auction_house_name', 'auction_sale', 'auction_lot', 'estimate', 
    'price_realized','price_realized_USD','bought_in',
    'living', 'years_after_death_of_auction', 'artist_age_at_auction',
    'artist_age_at_artwork_completion', 'artwork_age_at_auction','years_ago_of_auction'
]

auctions[new_col_order]

,artist_name,Nationality,Gender,birth_year,death_year,title,date,start_date,end_date,medium,...,estimate,price_realized,price_realized_USD,bought_in,living,years_after_death_of_auction,artist_age_at_auction,artist_age_at_artwork_completion,artwork_age_at_auction,years_ago_of_auction
0,Pablo Picasso,Spanish,Male,1881.0,1973.0,Fillette au bateau (Maya),1938,1938.0,1938.0,oil on canvas,...,NaN,NaN,NaN,NaN,0,50.0,NaN,57.0,85.0,0
1,Pablo Picasso,Spanish,Male,1881.0,1973.0,Homme assis,1969,1969.0,1969.0,oil on panel laid down on cradled panel,...,"HK\$8,200,000–HK\$12,800,000(est)","HK\$10,650,000• US\$1,363,489",1363489,NaN,0,49.0,NaN,88.0,53.0,1
2,Pablo Picasso,Spanish,Male,1881.0,1973.0,Buffalo Bill,1911,1911.0,1911.0,oil and sand on canvas,...,"US\$10,000,000–US\$15,000,000(est)","US\$12,412,500",12412500,NaN,0,49.0,NaN,30.0,111.0,1
3,Pablo Picasso,Spanish,Male,1881.0,1973.0,Homme à la moustache,1970,1970.0,1970.0,oil and oil stick on panel,...,"US\$4,000,000–US\$6,000,000(est)","US\$4,620,000",4620000,NaN,0,49.0,NaN,89.0,52.0,1
4,Pablo Picasso,Spanish,Male,1881.0,1973.0,Le peintre et son modèle,1964,1964.0,1964.0,oil and ripolin on canvas,...,"US\$8,000,000–US\$12,000,000(est)","US\$10,351,500",10351500,NaN,0,49.0,NaN,83.0,58.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53029,Francis Bacon,British,Male,1909.0,1992.0,Figure with 2 owls - Study after Velazquez Inn...,NaN,NaN,NaN,"oil, sand on canvas",...,NaN,"US\$1,200,000",1200000,NaN,1,NaN,78.0,NaN,NaN,36
53030,Francis Bacon,British,Male,1909.0,1992.0,Study for a portrait,1966,1966.0,1966.0,oil on canvas,...,NaN,"£380,000 • US\$611,800",611800,NaN,1,NaN,78.0,57.0,21.0,36
53031,Francis Bacon,British,Male,1909.0,1992.0,Portrait of George Dyer talking,1966,1966.0,1966.0,oil on canvas,...,NaN,"US\$1,300,000",1300000,NaN,1,NaN,78.0,57.0,21.0,36
53032,Francis Bacon,British,Male,1909.0,1992.0,Study for Portrait II,NaN,NaN,NaN,oil on canvas,...,NaN,"US\$1,600,000",1600000,NaN,1,NaN,78.0,NaN,NaN,36


In [63]:
auctions.to_csv('data/auction_data_cleaned.csv')